<a href="https://colab.research.google.com/github/thanapornIce/test/blob/main/%E0%B8%AA%E0%B8%AB%E0%B8%81%E0%B8%B4%E0%B8%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ข้อสอบประมวณผลภาพ
from google.colab import files
uploaded = files.upload()

import zipfile
import os
with zipfile.ZipFile("Model.zip", 'r') as zip_ref:
    zip_ref.extractall("Model")

os.listdir("Model")


Saving Model.zip to Model.zip


['Model']

In [ ]:
!pip install opencv-python lxml tensorflow

In [28]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17496, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 17496 (delta 2), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17496/17496), 16.54 MiB | 19.20 MiB/s, done.
Resolving deltas: 100% (11990/11990), done.
/content/yolov5/yolov5


In [29]:
import pandas as pd
import os

csv_path = "/content/gun.csv"
image_dir = "/content/images"  # ที่เก็บภาพทั้งหมด
label_dir = "/content/labels"  # จะเก็บ label txt ไฟล์

os.makedirs(label_dir, exist_ok=True)

df = pd.read_csv(csv_path)

# สร้าง mapping class → id
label_list = df['label_name'].unique().tolist()
label_map = {name: idx for idx, name in enumerate(label_list)}
print("Label map:", label_map)

# loop แปลง csv → YOLO txt
for image_name, group in df.groupby('image_name'):
    img_width = group['image_width'].values[0]
    img_height = group['image_height'].values[0]
    lines = []

    for _, row in group.iterrows():
        class_id = label_map[row['label_name']]
        x_center = (row['bbox_x'] + row['bbox_width'] / 2) / img_width
        y_center = (row['bbox_y'] + row['bbox_height'] / 2) / img_height
        width = row['bbox_width'] / img_width
        height = row['bbox_height'] / img_height
        lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # save as .txt
    txt_path = os.path.join(label_dir, os.path.splitext(image_name)[0] + ".txt")
    with open(txt_path, "w") as f:
        f.write("\n".join(lines))


Label map: {'gun': 0}


In [30]:
import os
import shutil
from sklearn.model_selection import train_test_split


image_dir = '/content/Model/Model/train/gun'
label_dir = '/content/labels'


base_dir = "/content/yolov5/dataset_split"
os.makedirs(f"{base_dir}/images/train", exist_ok=True)
os.makedirs(f"{base_dir}/images/val", exist_ok=True)
os.makedirs(f"{base_dir}/labels/train", exist_ok=True)
os.makedirs(f"{base_dir}/labels/val", exist_ok=True)


image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

def copy_files(files, split):
    for f in files:
        src_img = os.path.join(image_dir, f)
        dst_img = os.path.join(base_dir, f"images/{split}", f)
        shutil.copy2(src_img, dst_img)

        label_file = os.path.splitext(f)[0] + ".txt"
        src_lbl = os.path.join(label_dir, label_file)
        dst_lbl = os.path.join(base_dir, f"labels/{split}", label_file)

        if os.path.exists(src_lbl):
            shutil.copy2(src_lbl, dst_lbl)

copy_files(train_files, "train")
copy_files(val_files, "val")

print("แบ่ง train/val และคัดลอกเรียบร้อย")


แบ่ง train/val และคัดลอกเรียบร้อย


In [32]:
!cat /content/yolov5/data.yaml



train: /content/yolov5/dataset_split/images/train
val: /content/yolov5/dataset_split/images/val

nc: 3
names: ['gun', 'knife', 'bomb']


In [ ]:
!python train.py --data /content/yolov5/data.yaml --weights yolov5s.pt --epochs 30


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-02 08:48:52.183814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751446132.222387   13558 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751446132.234186   13558 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data.yaml, hyp=data/hyps

In [19]:
!python detect.py --weights runs/train/weapon_detector/weights/best.pt --img 640 --conf 0.25 --source /content/Model/Model/test/gun/08.png


detect: weights=['runs/train/weapon_detector/weights/best.pt'], source=/content/Model/Model/test/gun/08.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.6.0+cu124 CPU

Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 438, in <module>
    main(opt)
  File "/content/yolov5/detect.py", line 433, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/content/yolov5/detect.py", line 166